# Optical Flow


In [1]:
%matplotlib inline

import cv2
import numpy as np


###  Sparse optical flow

In this part you will calculate and visualize optical flow on a live video feed from a webcam.

First, find some good features to track, use either Harris corners, or Shi-Tomasi corners (`cv2.goodFeaturesToTrack`).

Then, you will track these features from frame to frame, and display their location on the live video feed. Use: `cv2.calcOpticalFlowPyrLK`.

As features tend to get lost during the time (check their returned status), it is advised to look for new features to track, from time to time (either every N frames, or when the features number falls below some level).

Try to add a visual trace of the motion. See for example: https://www.youtube.com/watch?v=E86NLzNbuL8


- API doc: https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html

- Example: http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html



In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.01,
                       minDistance = 5,
                       blockSize = 11 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (30,30),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

point_limit = int(len(p0) * 0.8)


while(1):
    # recalc point if there are too few
    if len(p0) < point_limit:
        ret, old_frame = cap.read()
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        p0 = cv2.goodFeaturesToTrack(old_gray, 100, 0.01, 10, None, None, 9)
    else:
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            # mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
        img = cv2.add(frame,mask)

        cv2.imshow('frame',img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

### Interactive tracking

In this part, we will add some interactive aspects to your previous implementation. 

First, the live video is shown without any features. 

Then, the user draws a region-of-interest (ROI) box on the video. Only the features that fall inside this region should be tracked from frame to frame. 

In every frame, plot the bounding box of the tracked features (using cv2.boundingRect).

Once again, since features tend to get lost over time, find some new features to track every now and then.

See for example: https://www.youtube.com/watch?v=kcrGkD2HOZs

Tutorial on handling mouse events in OpenCV: 
- http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_mouse_handling/py_mouse_handling.html



In [13]:
import cv2
import numpy as np

drawing = False # true if mouse is pressed
ix,iy = -1,-1

cap = cv2.VideoCapture(0)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.01,
                       minDistance = 5,
                       blockSize = 11 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (30,30),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing,mode
    global rect
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
        rect = ((ix,iy),(x,y))
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            rect = ((ix,iy),(x,y))

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rect = ((ix,iy),(x,y))
        
    
            

img = np.zeros((512,512,3), np.uint8)

# start x,y , finish x,y
rect = ((0,0),(0,0))

cv2.namedWindow('select ROI with mouse')
cv2.setMouseCallback('select ROI with mouse', draw_circle)

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

point_limit = int(len(p0) * 0.8)


while(True):
    ret, img = cap.read()
    # recalc point if there are too few
    if len(p0) < point_limit:
        old_frame = img.copy()
        old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
        p0 = cv2.goodFeaturesToTrack(old_gray, 100, 0.01, 10, None, None, 9)
    else:
        frame = img.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            
            # show only points inside the region of interest
            if((rect[0][0] <= a <= rect[1][0] or rect[0][0] >= a >= rect[1][0])
               and (rect[0][1] <= b <= rect[1][1] or rect[0][1] >= b >= rect[1][1])):
                img = cv2.circle(img, (a,b), 3, color[i].tolist(), -1)

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
    
    color_of_border = (drawing * 255, 255, drawing * 255)
    cv2.rectangle(img, (rect[0][0],rect[0][1]), (rect[1][0],rect[1][1]), color=color_of_border, thickness=2)
    # draw the last finished rectangle
    cv2.imshow('select ROI with mouse',img)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

## Good Luck!